In [1]:
import numpy as np
np.random.seed(5)

import functions_class as fx
import classx as cl
import log_reg_functions as lrf
import loaddata as ld


import time
import matplotlib.pyplot as plt

import pandas as pd
import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss, f1_score


# A ) 
Load data

In [2]:
x,y = ld.load_data(scaler='minmax')

xtrain,xtest,ytrain,ytest = train_test_split(x,y, stratify=y, test_size=0.25, random_state= 0, shuffle=True) 

xtrain,xval,ytrain,yval = train_test_split(xtrain,ytrain, stratify=ytrain, test_size=0.25, random_state= 0, shuffle=True) 
# do this because want our test to be seperate, first splitting into test and train, and then splitting traindata 
#into training and validation

nx_train, ny_train = xtrain.shape
nx_test, ny_test = xtest.shape


In [3]:
indices = np.where(y == 1)
indices_zero = np.where(y == 0)
datapoints = np.random.choice(indices_zero[0], size=y[indices[0]].shape[0], replace=False)

x_new = np.vstack((x[indices[0],:],x[datapoints,:]))
y_new = np.vstack((y[indices[0]],y[datapoints]))

xtrain = x_new
ytrain = y_new
nx_train, ny_train = xtrain.shape
nx_test, ny_test = xtest.shape



# B) 
Egen logistic regression med gradient descent.


In [4]:
eta = 0.1 # learning rate
doplot = False
doprint = False
Niteration = 10
beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_val=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

start = time.time()
for iter in range(Niteration):
    
    sig = lrf.sigmoid(xtrain@beta)
    gradient = lrf.gradient_ols(xtrain,ytrain,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(xtrain@beta,ytrain.T)
    cost_val = lrf.cost_log_ols(xval@beta,yval.T) # do this for testdata at the end. 
    #Log Loss function from sklearn
    if doprint:
        logloss=log_loss(ytrain, np.round(xtrain@beta), eps=1e-16, normalize=True)
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_val)
    if doplot:
        costvec.append(cost.ravel())
        costvec_val.append(cost_val.ravel())
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_val, 'r')
        plt.pause(1e-12)
plt.show()    
end = time.time()
print(end - start)
   
# et relevant plot er iterativt plot. 

0.05084800720214844


In [5]:
# making confusion matrix to check observed data with model predictions. 

predictions = xval@beta

sig_val = np.round(lrf.sigmoid(predictions))

cm = confusion_matrix(yval , sig_val.astype(int))
print(cm)

[[3209 1172]
 [ 957  287]]


# Accuracy. 
Både egen kode og tester med scikit. 

In [6]:

activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')
print(f1_score(ytrain, classes, 'F1-score'))

activation =lrf.sigmoid(xval@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==yval)/len(activation),'% Validation Accuracy')
print(f1_score(yval, classes, 'F1-score'))

activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')
print(f1_score(ytest, classes, 'F1-score'))




48.00331525015069 % Training Accuracy
0.3143566815697963
62.15111111111111 % Validation Accuracy
0.21235664076951535
61.373333333333335 % Test Accuracy
0.21554291903601405


# Egen logistisk regresjon med stokastisk gradient descent

In [7]:
eta = 0.1 # learning rate
doplot = False
doprint = False
Niteration = 1000
batch_size = 3000
batch_size_held_out = nx_train-batch_size

beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_test=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

indexes = np.arange(nx_train)


start = time.time()
for iter in range(Niteration):
    datapoints = np.random.choice(indexes, size=batch_size, replace=False)
    batch_x = xtrain[datapoints,:]
    batch_y = ytrain[datapoints]
    
    batch_x_held_out = np.delete(xtrain, datapoints, axis=0)
    batch_y_held_out = np.delete(ytrain, datapoints).reshape([batch_size_held_out,1])
    
    sig = lrf.sigmoid(batch_x@beta)
    gradient = lrf.gradient_ols(batch_x,batch_y,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(batch_x@beta,batch_y.T)
    cost_test = lrf.cost_log_ols(batch_x_held_out@beta,batch_y_held_out.T)
    #Log Loss function from sklearn
    if doprint:
        logloss=log_loss(batch_y, np.round(batch_x@beta), eps=1e-16, normalize=True)
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_test)
    if doplot:
        costvec.append(cost.ravel())
        costvec_test.append(cost_test.ravel())
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_test, 'r')
        plt.pause(1e-12)
plt.show()

end = time.time()
print(end - start)
 

7.225665092468262


In [8]:
activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')

activation =lrf.sigmoid(xval@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==yval)/len(activation),'% Validation Accuracy')

activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')


67.47287522603979 % Training Accuracy
73.99111111111111 % Validation Accuracy
73.65333333333334 % Test Accuracy


In [9]:

model = LogisticRegression()
model.fit(xtrain, ytrain)
predicted_classes = model.predict(xtrain)
accuracy = accuracy_score(ytrain.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytrain, predicted_classes)

print(accuracy, '% Training Accuracy')

predicted_classes = model.predict(xtest)
accuracy = accuracy_score(ytest.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytest, predicted_classes)

print(accuracy, '% Test Accuracy')

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


70.46413502109705 % Training Accuracy
77.68 % Test Accuracy


# C) 
Neural Network. 

In [10]:
#X_data_full = X_data 
#Y_data_full = Y_data
 

n_inputs, n_features = x.shape

#epochs = epochs
#batch_size = batch_size
#iterations = self.n_inputs // self.batch_size
#eta = eta
#lmbd = lmbd


n_hidden_neurons=50  
n_categories=2
epochs=100
batch_size=80
eta=0.0001
lmbd=0.1

# Creating weights and bias.  

hidden_weights = np.random.randn(n_features, n_hidden_neurons)
hidden_bias = np.zeros(n_hidden_neurons) + 0.01

output_weights = np.random.randn(n_hidden_neurons, n_categories)
output_bias = np.zeros(n_categories) + 0.01

In [11]:
#Feed forward
z_h = np.matmul(x, hidden_weights) + hidden_bias
a_h = lrf.sigmoid(z_h)

z_o = np.matmul(a_h, output_weights) + output_bias

exp_term = np.exp(z_o)
probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

In [15]:
#Backpropagation

#a_h, probabilities = feed_forward_train(X)
    
# error in the output layer
error_output = probabilities - y
# error in the hidden layer
error_hidden = np.matmul(error_output, output_weights.T) * a_h * (1 - a_h)
    
# gradients for the output layer
output_weights_gradient = np.matmul(a_h.T, error_output)
output_bias_gradient = np.sum(error_output, axis=0)
    
# gradient for the hidden layer
hidden_weights_gradient = np.matmul(x.T, error_hidden)
hidden_bias_gradient = np.sum(error_hidden, axis=0)

if lmbd > 0.0:
        output_weights_gradient += lmbd * output_weights
        hidden_weights_gradient += lmbd * hidden_weights

        output_weights -= eta * output_weights_gradient
        output_bias -= eta * output_bias_gradient
        hidden_weights -= eta * hidden_weights_gradient
        hidden_bias -= eta * hidden_bias_gradient

Forsøk på Neural network: vil ha activation funksjon i flere lag, men må avslutte med sigmoid. Bruke ReLu. 

In [ ]:
def forward_activation(x): 
    if self.activation_function == "sigmoid":
      return 1.0/(1.0 + np.exp(-X))
    elif self.activation_function == "relu":
      return np.maximum(0,X)
      
def grad_activation(x):
    if activation_function == "sigmoid":
      return X*(1-X) 
    elif activation_function == "relu":
      return 1.0*(X>0)


def softmax(x):
    exps = np.exp(X)
    return exps / np.sum(exps, axis=1).reshape(-1,1)